In [2]:
# Google Colaboratory用 レストラン検索プログラム

# ライブラリのインポート（requestsは標準搭載）
import requests
import getpass
from pprint import pprint

def get_station_coordinates(station_name, api_key):
    """
    駅名から緯度経度を取得する関数
    """
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json"

    params = {
        'address': station_name,
        'key': api_key
    }

    response = requests.get(geocode_url, params=params)
    data = response.json()

    # デバッグ用（必要に応じてコメント解除）
    # print("\n=== Geocoding API Response ===")
    # pprint(data)

    if data['status'] == 'OK':
        location = data['results'][0]['geometry']['location']
        latitude = location['lat']
        longitude = location['lng']
        return latitude, longitude
    else:
        print(f"エラー: 駅の位置情報が取得できませんでした。ステータス: {data['status']}")
        return None, None

def get_nearby_restaurants(latitude, longitude, api_key, radius=500):
    """
    指定された緯度経度の周辺レストラン情報を取得する関数
    """
    places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    params = {
        'location': f'{latitude},{longitude}',
        'radius': radius,
        'type': 'restaurant',
        'language': 'ja',
        'key': api_key
    }

    response = requests.get(places_url, params=params)
    data = response.json()

    # デバッグ用（必要に応じてコメント解除）
    # print("\n=== Places API Response ===")
    # pprint(data)

    return data

def display_restaurant_info(restaurants_data):
    """
    レストラン情報を整形して出力する関数
    """
    if restaurants_data['status'] != 'OK':
        print(f"エラー: レストラン情報が取得できませんでした。ステータス: {restaurants_data['status']}")
        return

    results = restaurants_data.get('results', [])

    if not results:
        print("指定された範囲内にレストランが見つかりませんでした。")
        return

    print(f"\n{'='*60}")
    print(f"見つかったレストラン数: {len(results)}件")
    print(f"{'='*60}\n")

    for idx, restaurant in enumerate(results, 1):
        name = restaurant.get('name', '名前不明')
        rating = restaurant.get('rating', '評価なし')
        address = restaurant.get('vicinity', '住所不明')

        print(f"【レストラン {idx}】")
        print(f"  レストラン名: {name}")
        print(f"  評価: {rating}")
        print(f"  住所: {address}")
        print(f"{'-'*60}\n")

def main():
    """
    メイン処理
    """
    print("=== Google Maps API レストラン検索プログラム ===\n")

    # APIキーの入力（Google Colabでは入力フォームが表示される）
    api_key = getpass.getpass("Google Maps APIキーを入力してください: ")

    # 駅名の入力（Google Colabでは入力フォームが表示される）
    station_name = input("駅名を入力してください（例: 渋谷駅）: ")

    print(f"\n{station_name}の位置情報を取得中...")

    # 駅の緯度経度を取得
    latitude, longitude = get_station_coordinates(station_name, api_key)

    if latitude is None or longitude is None:
        print("処理を終了します。")
        return

    print(f"✓ 緯度: {latitude}, 経度: {longitude}")
    print(f"\n半径500メートル以内のレストランを検索中...")

    # レストラン情報を取得
    restaurants_data = get_nearby_restaurants(latitude, longitude, api_key)

    # レストラン情報を表示
    display_restaurant_info(restaurants_data)

    print("\n処理が完了しました！")

# Google Colab用：このセルを実行
if __name__ == "__main__":
    main()


=== Google Maps API レストラン検索プログラム ===

Google Maps APIキーを入力してください: ··········
駅名を入力してください（例: 渋谷駅）: 寝屋川市駅

寝屋川市駅の位置情報を取得中...
✓ 緯度: 34.7639892, 経度: 135.6206697

半径500メートル以内のレストランを検索中...

見つかったレストラン数: 20件

【レストラン 1】
  レストラン名: 麺屋 一照
  評価: 3.9
  住所: 寝屋川市大利町２０−１９
------------------------------------------------------------

【レストラン 2】
  レストラン名: 本格四川料理 華洋
  評価: 4
  住所: 寝屋川市早子町１９−２
------------------------------------------------------------

【レストラン 3】
  レストラン名: てん
  評価: 4.3
  住所: 寝屋川市大利町１２−７４−２
------------------------------------------------------------

【レストラン 4】
  レストラン名: SUSHI BAR 八坂
  評価: 4
  住所: 寝屋川市, ２３-3 八坂町, 大阪市
------------------------------------------------------------

【レストラン 5】
  レストラン名: 伊仏食堂 DAI
  評価: 4.4
  住所: 寝屋川市八坂町１４−１６ ジョイフル寝屋川１F
------------------------------------------------------------

【レストラン 6】
  レストラン名: まどいせん
  評価: 3.8
  住所: 寝屋川市八坂町１４−２ 雅ビル
------------------------------------------------------------

【レストラン 7】
  レストラン名: 黒毛和牛焼肉 牛極 雅楽
  評価: 3.9
  住所: 寝屋川市東大利町２−１１ プチポート